# 第二次检测修复字符串

In [7]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
original_language = "English"
target_language = "Vietnamese"
def generate_text(index, data):
    try:
        if str(data['备注']).strip() == 'SPECIAL VALUE' or str(data['备注']).strip() == 'SPECIAL VALUE, INCLUDE UPPERCASE LETTER' or str(data['备注']).strip() == '':
            return index, str(data['待翻译(译)']).strip()
        if str(data['参考语言(英文)']).strip().lower() != str(data['简体中文(源)']).strip().lower() and str(data['参考语言(英文)']).strip().lower() == str(data['待翻译(译)']).strip().lower():
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": f"You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}. Preserving its formatting without adding extra content."},
                    {"role": "user", "content": str(data['参考语言(英文)'])}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
        if str(data['参考语言(英文)']).strip().lower() == str(data['简体中文(源)']).strip().lower():
            return index, str(data['参考语言(英文)']).strip()
        else:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": "Translate English: The received L/G %s is not submitted \nTo Spanish: El L/G recibido no se ha enviado. \nPlease fix the translate error, because %s is missing, translate it directly and without adding any extra content."},
                    {"role": "assistant", "content": "El L/G recibido %s no se ha enviado."},
                    {"role": "user", "content": f"Translate {original_language}: {str(data['参考语言(英文)'])} \nTo {target_language}: {str(data['待翻译(译)'])} \n Please fix the translate error, Because {str(data['备注'])}. translate it directly and without adding any extra content."}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
    except:
        return index, str(data['待翻译(译)']).strip()

In [8]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, f'待翻译(译)'] = output

to_fix['备注'] = ""
# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越_processed.xlsx", index=False)

100%|██████████| 124202/124202 [02:26<00:00, 849.76it/s] 


# 文件合并

In [2]:
import pandas as pd

original = "英"
target = "印尼"
# 定义要合并的 Excel 文件列表
excel_files = [
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_1_10000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_10001_20000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_20001_30000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_30001_40000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_40001_50000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_50001_60000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_60001_70000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_70001_80000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_80001_90000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_90001_100000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_100001_110000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_110001_120000.xlsx",
    f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}_120001_130000.xlsx",
]

# 读取并合并所有 Excel 文件
df_list = [pd.read_excel(file) for file in excel_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 保存合并后的 DataFrame 到新的 Excel 文件
merged_df.to_excel(f"D:\\Projects\\ai-translator\\special value\\multilangInitData YS全量词条 20241119 Special Value_{original}_{target}.xlsx", index=False)
